In [1]:
# Upgrade dependencies
! pip install -r ../requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import boto3
import os
from os import path
import pandas as pd
!pip install torchtext
import re, time
import numpy as np
import torch, torchtext

from os import path
from collections import Counter
from torch import nn, optim
from torch.nn import BCEWithLogitsLoss
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from torchtext.vocab import GloVe
GloVe.url['6B'] = 'https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip'

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
train_df = pd.read_csv('../data/train_complete.csv', encoding='utf-8', header=0)
train_df.head()

,tweet,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2


In [4]:
lines = []
with open('../data/test_text.txt') as f:
    lines = [line.rstrip('\n') for line in f]
print(len(lines))
test_df = pd.DataFrame (lines, columns = ['tweet'])
test_df.describe()

12284


,tweet
count,12284
unique,12284
top,hey PA can you um let me register for a medica...
freq,1


In [5]:
class_id_distribution = train_df['label'].value_counts()
print(class_id_distribution)

1    21542
2    18668
0     7405
Name: label, dtype: int64


In [6]:
print("Fixing missing values...")
train_df["tweet"].fillna("", inplace=True)

print("Splitting data into training and validation...")
X_train, X_val, y_train, y_val = train_test_split(
    train_df["tweet"].tolist(),
    train_df["label"].tolist(),
    test_size=0.10,
    shuffle=True,
    random_state=324,
)

Fixing missing values...
Splitting data into training and validation...


In [10]:
# Making the samples equal for each class
X_train_sample_class_0 = X_train[y_train == 0]
X_train_sample_class_1 = X_train[y_train == 1]
X_train_sample_class_2 = X_train[y_train == 2]

from sklearn.utils import resample
X_train_sample_class_1 = resample(X_train_sample_class_1,
             replace=True,
             n_samples=len(X_train_sample_class_0)+3000,
             random_state=42)
X_train_sample_class_2 = resample(X_train_sample_class_2,
             replace=True,
             n_samples=len(X_train_sample_class_0)+3000,
             random_state=42)
y_train_sample_class_0 = np.zeros(len(X_train_sample_class_0))
y_train_sample_class_1 = np.ones(len(X_train_sample_class_1))
y_train_sample_class_2 = np.repeat(2, len(X_train_sample_class_2))
# print(X_train_sample_class_1.shape)
# print(X_train_sample_class_2.shape)

X_train = pd.concat([pd.concat([X_train_sample_class_0, X_train_sample_class_1]), X_train_sample_class_2])
y_train = np.concatenate((np.concatenate((y_train_sample_class_0, y_train_sample_class_1), axis=0), y_train_sample_class_2), axis=0)
print(X_train.shape)
print(y_train.shape)

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [ ]:
for index in range(len(X_train)):
    sent = X_train[index]
    if index==0:
        print(X_train[index])
    # Lowercase
    sent = sent.lower()
    # Remove leading/trailing whitespace
    sent = sent.strip()
    # Remove extra space and tabs
    sent = re.sub("\s+", " ", sent)
    # Remove HTML tags/markups:
    sent = re.compile("<.*?>").sub("", sent)
    # initializing punctuations string
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in sent:
        if ele in punc:
            sent = sent.replace(ele, "")
            
    sent = ''.join([i for i in sent if not i.isdigit()])
    
    X_train[index] = sent
    if index==0:
        print(X_train[index])

### Text transformation
We will apply the following processes here:
1. Creating a vocabulary
2. Text transformation

In [22]:
tokenizer = get_tokenizer("basic_english")
counter = Counter()
for line in X_train:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=2) #min_freq>1 for skipping misspelled words

TypeError: __init__() got an unexpected keyword argument 'min_freq'

In [23]:
# Some examples
print(f"'home' -> {vocab['home']}")
print(f"'wash' -> {vocab['wash']}")
# unknown word (assume from test set)
print(f"'fhshbasdhb' -> {vocab['fhshbasdhb']}")

NameError: name 'vocab' is not defined

In [24]:
# Let's create a mapper to transform our text data
text_transform_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

In [ ]:
print(f"Before transform:\t{X_train[7]}")
print(f"After transform:\t{text_transform_pipeline(X_train[7])}")

In [ ]:
def pad_features(reviews_split, seq_length):
    print(reviews_split)
    # Transform the text
    # use the dict to tokenize each review in reviews_split
    # store the tokenized reviews in reviews_ints
    reviews_ints = []
    for review in reviews_split:
        reviews_ints.append(text_transform_pipeline(review))
    
    # getting the correct rows x cols shape
    features = np.ones((len(reviews_ints), seq_length), dtype=int)
    
    # for each review, I grab that review
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return torch.tensor(features, dtype=torch.int64)

In [ ]:
for text in X_train[9:11]:
    print(f"Text: {text}\n")
    print(f"Original length of the text: {len(text)}\n")
    tt = pad_features([text], seq_length=100)
    print(f"Transformed text: \n{tt}\n")
    print(f"Shape of transformed text: {tt.shape}\n")

In [ ]:
max_len = 200
batch_size = 64

# Pass transformed and padded data to dataset
# Create data loaders
train_dataset = TensorDataset(
    pad_features(X_train, max_len), torch.tensor(y_train, dtype=torch.float32)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = TensorDataset(pad_features(X_val, max_len), torch.tensor(y_val, dtype=torch.float32))
val_loader = DataLoader(val_dataset, batch_size=batch_size)

### Using pre-trained GloVe word embeddings

In this example, we will use GloVe word vectors. `name='6B'` `dim=200` gives us 6 billion words/phrases vectors. Each word vector has 200 numbers in it. The following code shows how to get the word vectors and create an embedding matrix from them. We will connect our vocabulary indexes to the GloVe embedding with the `get_vecs_by_tokens()` function.

In [ ]:
glove = GloVe(name="6B", dim=200)
embedding_matrix = glove.get_vecs_by_tokens(vocab.itos)

In [ ]:
# Size of the state vectors
hidden_size = 128

# General NN training parameters
learning_rate = 0.0001
epochs = 25

# Embedding vector and vocabulary sizes
embed_size = 200  # glove.6B.300d.txt
vocab_size = len(vocab.itos)

Our model is made of these layers:

- Embedding layer: This is where our words/tokens are mapped to word vectors.
- RNN layer: We are using a simple RNN model. We stack 2 RNN layers in this example.
- Linear layer: A linear layer with a single neuron is used to output the prediction.

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=1)
        self.rnn = nn.RNN(
            embed_size, hidden_size, num_layers=num_layers, batch_first=True
        )

        self.linear = nn.Linear(hidden_size, 1)
        self.act = nn.Sigmoid()

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # Call the RNN layer
        outputs, _ = self.rnn(embeddings)
        
        # Output shape after RNN: (batch_size, max_len, hidden_size)
        # Get the output from the last time step with outputs[:, -1, :] below
        # The output shape becomes: (batch_size, 1, hidden_size)
        # Send it to the linear layer
        outs = self.linear(outputs[:, -1, :])
        return self.act(outs)
    
# Initialize the weights
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.RNN:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])

In [ ]:
# Our architecture with 2 RNN layers
model = Net(vocab_size, embed_size, hidden_size, num_layers=2)

# We set the embedding layer's parameters from GloVe
model.embedding.weight.data.copy_(embedding_matrix)
# We won't change/train the embedding layer
model.embedding.weight.requires_grad = False

In [ ]:
# Setting our trainer
trainer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# We will use Binary Cross-entropy loss
# reduction="sum" sums the losses for given output and target
cross_ent_loss = nn.BCELoss(reduction="sum")

In [ ]:
# Get the compute device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.apply(init_weights)
model.to(device)

for epoch in range(epochs):
    start = time.time()
    training_loss = 0
    val_loss = 0
    # Training loop, train the network
    for data, target in train_loader:
        trainer.zero_grad()
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        L = cross_ent_loss(output, target.unsqueeze(1))
        training_loss += L.item()
        L.backward()
        trainer.step()

    # Validate the network, no training (no weight update)
    for data, target in val_loader:
        val_predictions = model(data.to(device))
        L = cross_ent_loss(val_predictions, target.to(device).unsqueeze(1))
        val_loss += L.item()

    # Let's take the average losses
    training_loss = training_loss / len(y_train)
    val_loss = val_loss / len(y_val)

    end = time.time()
    print(
        f"Epoch {epoch}. Train_loss {training_loss:.4f}. Val_loss {val_loss:.4f}. Seconds {end-start:.3f}"
    )

## 3. Make predictions on your test dataset

In [28]:
# Implement this
val_predictions = []
for data, target in val_loader:
    val_preds = model(data.to(device))
    val_predictions.extend(
        [np.rint(val_pred)[0] for val_pred in val_preds.detach().cpu().numpy()]
    )
print(val_predictions[:10])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [29]:
# Use the fitted pipeline to make predictions on the validation dataset
print(confusion_matrix(y_val, val_predictions))
print(classification_report(y_val, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val, val_predictions))

[[   0  730    0]
 [   0 2211    0]
 [   0 1821    0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       730
           1       0.46      1.00      0.63      2211
           2       0.00      0.00      0.00      1821

    accuracy                           0.46      4762
   macro avg       0.15      0.33      0.21      4762
weighted avg       0.22      0.46      0.29      4762

Accuracy (validation): 0.46430071398572026


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier